In [13]:
import numpy as np
import pandas as pd
import matplotlib
import os

In [14]:
#delete ps and table files
rootdir='C:\\Users\\k1222742\\Dropbox\\IoP\\Preclinical\\NAC\\NAC5_NLGN3\\Results'
for subdirs, dirs,  files in os.walk(rootdir):
    for file in files:
        name= os.path.join(subdirs, file)
        if  not ".csv" in name:
            try:
                os.remove(os.path.join(subdirs, file))
            except: 
                print("already gone")
        

In [23]:
#loop through directories and get info from csv file. 
mrs=pd.DataFrame([])
for subdirs, dirs,  files in os.walk(rootdir):
    for file in files:
        scan_id=  os.path.join(subdirs, file)[65:76]
        tmp= pd.read_csv(os.path.join(subdirs, file))
        tmp['scan_id']="NAC5_"+scan_id
        mrs=mrs.append(tmp)
mrs=mrs.set_index('scan_id')
#only keep variables i want: cr, glu, glx, gaba, gsh
mrs_good= mrs[[' Cr',' Cr %SD', ' PCr',' PCr %SD',' Glu', ' Glu/Cr+PCr', ' Glu %SD',  ' GABA', ' GABA %SD', ' GABA/Cr+PCr',' GSH', ' GSH %SD',' GSH/Cr+PCr', ' NAA', ' NAA %SD', ' NAA/Cr+PCr', ' Ins', ' Ins %SD', ' Ins/Cr+PCr']]
#df.rename(columns={' Cr':'Cr',' Cr %SD':'CrSD', ' PCr':'PCr',' PCr %SD',' Glu', ' Glu/Cr+PCr', ' Glu %SD',  ' GABA', ' GABA %SD', ' GABA/Cr+PCr',' GSH', ' GSH %SD',' GSH/Cr+PCr'}, inplace=True)
mrs_good.rename(columns=lambda x: x.replace(" ", ""), inplace=True)
mrs_good.rename(columns=lambda x: x.replace("%", ""), inplace=True)
mrs_good.rename(columns=lambda x: x.replace("/", ""), inplace=True)
mrs_good.rename(columns=lambda x: x.replace("+", ""), inplace=True)

print(mrs_good)
#list(mrs_good.columns.values)

                      Cr  CrSD     PCr  PCrSD     Glu  GluCrPCr  GluSD   GABA  \
scan_id                                                                         
NAC5_20160125_01   9.470    16   5.687     27  16.740     1.104     12  3.148   
NAC5_20160125_02  11.066    26   5.247     60  23.815     1.460     11  3.263   
NAC5_20160125_03   8.542    26   4.989     46  21.237     1.570     11  2.631   
NAC5_20160125_04  11.623    15   2.985     58  20.409     1.397     10  2.378   
NAC5_20160126_01   5.413    23   8.971     15  20.385     1.417     11  1.969   
NAC5_20160126_02   7.739    14   6.364     17  19.326     1.370      9  3.358   
NAC5_20160126_03   9.420    14   6.512     20  15.034     0.944     13  2.989   
NAC5_20160126_04   5.946    30   8.423     22  19.977     1.390     12  2.182   
NAC5_20160126_05   6.167    25   8.352     19  17.504     1.206     11  2.564   
NAC5_20160127_01   8.852    20   5.170     35  19.510     1.391     11  1.253   
NAC5_20160127_02   6.748    

C:\Users\k1222742\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [24]:
# get info from scan sheet and merge df
grid=pd.read_csv('C:\\Users\\k1222742\\Dropbox\\IoP\\Preclinical\\NAC\\NAC5_NLGN3\\NAC5_Grid_Clean.csv',encoding = "ISO-8859-1")
grid=grid.set_index('SCAN_ID')
grid.Gen=grid.Gen.str.strip()
grid.Ttt=grid.Ttt.str.strip()



In [25]:
#Merge dfs
Data=grid.merge(mrs_good,right_index=True,left_index=True)
Data['CrPCr']=Data.Cr+Data.PCr
#print(Data)
Data.to_csv('Data.csv')



In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

formula = 'GluCrPCr ~ C(Gen) + C(Ttt) + C(Gen):C(Ttt)'
model = ols(formula, Data[(Data.GluSD<=20) & (Data.wrong_coil==0)].fit()
bla=statsmodels.stats.anova.anova_lm(model, typ=2)
print(aov_table)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import sem

#calc mean and sem
Data_sumstats = (Data[Data['Glu%SD']<20].groupby(["Gen", "Ttt"]).Glu.agg(["mean", sem]).reset_index())
print(Data_sumstats)

SalMeans = np.array(Data_sumstats[Data_sumstats.Ttt=='Saline']['mean'])
SalSem = np.array(Data_sumstats[Data_sumstats.Ttt=='Saline']['sem'])
NACMeans = np.array(Data_sumstats[Data_sumstats.Ttt=='NAC']['mean'])
NACSem = np.array(Data_sumstats[Data_sumstats.Ttt=='NAC']['sem'])
print(SalMeans)
#plot
N=2
ind = np.arange(N)  # the x locations for the groups
width = 0.35       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, SalMeans, width, color='r', yerr=SalSem)
rects2 = ax.bar(ind+width, NACMeans, width, color='b', yerr=NACSem)

# add some text for labels, title and axes ticks
ax.set_ylabel('Glutamate')
#ax.set_title('Scores by group and gender')
ax.set_xticks(ind + width)
ax.set_xticklabels(('KO', 'WT'))

ax.legend((rects1[0], rects2[0]), ('Saline', 'NAC'))

plt.show()

http://www.marsja.se/three-ways-to-carry-out-2-way-anova-with-python/